# DFT calculations in DFTK

In [ ]:
# Activate source directory of DFTK
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using DFTK

### Parameters

In [ ]:
kgrid = [3, 3, 3]    # k-Point grid
Ecut = 15            # Kinetic energy cutoff in Hartree
Tsmear = 0.01        # Smearing temperature in Hartree (for metals)
T = Float64;         # Floating point type for computations

### Setting up lattices and structures

In [ ]:
# Build Silicon (an Insulator)

a = 10.263141334305942  # Silicon lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]
n_bands = 6

model = model_dft(lattice, :lda_xc_teter93, atoms);

or

In [ ]:
n_electrons = sum(length(pos) * n_elec_valence(spec) for (spec, pos) in atoms)    
model = Model(Array{T}(lattice); n_electrons=n_electrons, atoms=atoms,
              external=term_external(atoms),    
              nonlocal=term_nonlocal(atoms),    
              hartree=term_hartree(),    
              xc=term_xc(:lda_x, :lda_c_vwn));

or

In [ ]:
# Build graphite (a Metal)
# Note: This is not exactly the minimum-energy structure

ÅtoBohr = 1.8897261246257702  # Convert Ångström to bohr
a = 1.228ÅtoBohr   
b = 2.12695839ÅtoBohr
c = 7ÅtoBohr
lattice = [[a a 0]; [-b b 0]; [0 0 c]]
C = ElementPsp(:C, psp=load_psp("hgh/lda/C-q4"))
atoms = (C => [[0, 0, 1/4], [0, 0, 3/4],
                     [1/3, 2/3, 1/4], [2/3, 1/3, 3/4]], )
n_bands = 15

model = model_dft(Array{T}(lattice), :lda_xc_teter93, atoms;
                  temperature=Tsmear, smearing=Smearing.MethfesselPaxton2());

### Running an SCF computation

In [ ]:
# Discretise the model
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid);

In [ ]:
# Build self-consistent Hamiltonian

ham = Hamiltonian(basis, guess_density(basis, atoms))
scfres = self_consistent_field(ham, n_bands, tol=1e-6, mixing=KerkerMixing())
print_energies(scfres.energies)

println("\nkpt   occupation")
for ik in 1:length(basis.kpoints)
    println("   $ik  $(scfres.occupation[ik])")
end

### Post-process SCF results

In [ ]:
# Compute bandstructure
plot_bandstructure(scfres, n_bands)

In [ ]:
# Plot of the density versus the norm of the reciprocal lattice points
Gabs = vec([sum(abs, G) for G in G_vectors(basis)])
PyPlot.semilogy(Gabs, vec(abs.(scfres.ρ.fourier)), ".")

In [ ]:
# Plot the density of state
εs = collect(-0.3:0.005:0.3)
doses = [DFTK.DOS(ε + scfres.εF, basis, scfres.orben,
                  T=0.03, smearing=Smearing.Gaussian()) for ε in εs]
PyPlot.plot(εs, doses)